In [1]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!pip3 install tokenizers -U
!pip3 install transformers -U


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.1 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 82.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 107.1 MB/s eta 0:00:00m eta 0:00:01
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: python-debian 0.1.43ubuntu1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-debian or contact the author to suggest that they release a v

In [2]:
import torch  
from transformers import AutoTokenizer, AutoModelForCausalLM  
import jsonlines

/home/mohitkhatwani/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Load the tokenizer and model from Hugging Face  
 
model_id = "meta-llama/Meta-Llama-3-8B"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float32,
)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.56it/s]


In [6]:
# Save to disk  
output_path = "golden_data_llama3-8b.jsonl"   
    
        
# Your prompt text  
prompt_texts = ["I love to", "Today is a", "What is the"  ]
all_data_to_save = []


for prompt_text in prompt_texts:
    # Encode the prompt text  
    input_ids = tokenizer.encode(prompt_text, return_tensors='pt')  

    # Get the logits for the prompt + completion  
    with torch.no_grad():  
        outputs = model(input_ids)  
        logits = outputs.logits  

        # Convert logits to fp32  
        logits = logits.cpu().numpy().astype('float32')  

        # Prepare data to be saved  
        data_to_save = {  
            "prompt": prompt_text,  
            "tokens": input_ids.tolist()[0],  
            "logits": logits.tolist()[0]  # Convert numpy array to list for JSON serialization  
        }  
        all_data_to_save.append(data_to_save)
    
with jsonlines.open(output_path,'w') as f:    
    f.write_all(all_data_to_save)



print(f"Data saved to {output_path}")  


Data saved to golden_data_llama3-8b.jsonl
